<a href="https://colab.research.google.com/github/rchavarria3007/ml-ops-pavani/blob/main/feature-engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ==============================================================================
# CÉLULA 1: Instalação de Bibliotecas
# ==============================================================================
!pip install category_encoders
print("Biblioteca 'category_encoders' instalada.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 2.7 MB/s eta 0:00:00
Biblioteca 'category_encoders' instalada.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:

# ==============================================================================
# Setup e Carregamento de Dados
# ==============================================================================
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures, LabelEncoder
import category_encoders as ce
from google.colab import files
import io

# Configuração para exibir todas as colunas do pandas
pd.set_option('display.max_columns', None)

# Carregamento do dataset
try:
    file_name = "/content/drive/MyDrive/dev/datascienceexp/bz_hr_attr_features.csv"
    bz_hr_attr_features = pd.read_csv(file_name)
    print(f"\nDataset '{file_name}' carregado com sucesso!")
    print("Dimensões do dataset:", bz_hr_attr_features.head())
except StopIteration:
    print("\nNenhum arquivo foi enviado.")
    bz_hr_attr_features = None
except Exception as e:
    print(f"Ocorreu um erro ao carregar o arquivo: {e}")
    bz_hr_attr_features = None


# ==============================================================================
# Criação de Features Base para Transformação
# ==============================================================================
if bz_hr_attr_features is not None:
    print("\n--- Criando Features Base ---")
    # 1. Índice de Burnout
    bz_hr_attr_features['OverTime_num'] = bz_hr_attr_features['OverTime'].apply(lambda x: 1 if x == 'Yes' else 0)
    bz_hr_attr_features['IndiceDeBurnout'] = (5 - bz_hr_attr_features['WorkLifeBalance']) + \
      (5 - bz_hr_attr_features['JobSatisfaction']) + \
      bz_hr_attr_features['OverTime_num']

    # 2. Índice de Satisfação Geral
    bz_hr_attr_features['IndiceDeSatisfacaoGeral'] = (bz_hr_attr_features['JobSatisfaction'] + \
                                                      bz_hr_attr_features['EnvironmentSatisfaction'] + \
                                                      bz_hr_attr_features['RelationshipSatisfaction']) / 3

    print("Features base 'IndiceDeBurnout' e 'IndiceDeSatisfacaoGeral' criadas.")
    print(bz_hr_attr_features[['IndiceDeBurnout', 'IndiceDeSatisfacaoGeral']].head())




Dataset '/content/drive/MyDrive/dev/datascienceexp/bz_hr_attr_features.csv' carregado com sucesso!
Dimensões do dataset:    EmployeeNumber MaritalStatus  JobLevel  JobInvolvement  YearsAtCompany  \
0               1        Single         2               3               6   
1               2       Married         2               2              10   
2               4        Single         1               2               0   
3               5       Married         1               3               8   
4               7       Married         1               3               2   

   MonthlyIncome  StockOptionLevel  JobSatisfaction  EnvironmentSatisfaction  \
0           5993                 0                4                        2   
1           5130                 1                2                        3   
2           2090                 0                3                        4   
3           2909                 0                3                        4   
4           346

In [11]:

# ==============================================================================
# CÉLULA 4: Técnica Avançada 1: Geração de Features Polinomiais
# ==============================================================================
if bz_hr_attr_features is not None:
    print("\n--- Gerando Features Polinomiais ---")
    features_to_transform = ['MonthlyIncome', 'YearsAtCompany', 'IndiceDeBurnout', 'IndiceDeSatisfacaoGeral']
    poly_data = bz_hr_attr_features[features_to_transform]

    poly = PolynomialFeatures(degree=2, include_bias=False)
    poly_features = poly.fit_transform(poly_data)
    poly_feature_names = [name.replace(' ', '_X_') for name in poly.get_feature_names_out(features_to_transform)]
    df_poly = pd.DataFrame(poly_features, columns=poly_feature_names)

    print("Novas Features Polinomiais Geradas (amostra):")
    print(df_poly.head())


# ==============================================================================
# CÉLULA 5: Técnica Avançada 2: Geração de Features com Target Encoding
# ==============================================================================
if bz_hr_attr_features is not None:
    print("\n--- Gerando Features com Target Encoding ---")
    le = LabelEncoder()
    bz_hr_attr_features['Attrition_encoded'] = le.fit_transform(bz_hr_attr_features['Attrition'])

    categorical_features_to_encode = ['MaritalStatus']
    target_encoder = ce.TargetEncoder(cols=categorical_features_to_encode)
    df_target_encoded = target_encoder.fit_transform(bz_hr_attr_features[categorical_features_to_encode], bz_hr_attr_features['Attrition_encoded'])
    df_target_encoded = df_target_encoded.rename(columns={col: f'{col}_TargetEncoded' for col in df_target_encoded.columns})

    print("Novas Features com Target Encoding (amostra):")
    print(df_target_encoded.head())


# ==============================================================================
# CÉLULA 6: Consolidação e Seleção das 10 Features Avançadas Finais
# ==============================================================================
if bz_hr_attr_features is not None:
    print("\n--- Consolidando e Selecionando as 10 Features Finais ---")
    df_advanced_features = pd.concat([df_poly, df_target_encoded], axis=1)

    final_10_features_list = [
        # Target Encodings
        'MaritalStatus_TargetEncoded',
        # Potências
        'IndiceDeBurnout^2',
        'MonthlyIncome^2',
        # Interações
        'MonthlyIncome_X_IndiceDeBurnout',
        'YearsAtCompany_X_IndiceDeBurnout',
        'MonthlyIncome_X_IndiceDeSatisfacaoGeral',
        'YearsAtCompany_X_IndiceDeSatisfacaoGeral',
        'IndiceDeBurnout_X_IndiceDeSatisfacaoGeral'
    ]

    df_final_features = df_advanced_features[final_10_features_list]

    print("\nDataFrame Final com as 10 Features Avançadas Selecionadas:")
    print(df_final_features.info())
    print("\nAmostra das 10 Features Finais:")
    print(df_final_features.head())


--- Gerando Features Polinomiais ---
Novas Features Polinomiais Geradas (amostra):
   MonthlyIncome  YearsAtCompany  IndiceDeBurnout  IndiceDeSatisfacaoGeral  \
0         5993.0             6.0              6.0                 2.333333   
1         5130.0            10.0              5.0                 3.000000   
2         2090.0             0.0              5.0                 3.000000   
3         2909.0             8.0              5.0                 3.333333   
4         3468.0             2.0              5.0                 2.333333   

   MonthlyIncome^2  MonthlyIncome_X_YearsAtCompany  \
0       35916049.0                         35958.0   
1       26316900.0                         51300.0   
2        4368100.0                             0.0   
3        8462281.0                         23272.0   
4       12027024.0                          6936.0   

   MonthlyIncome_X_IndiceDeBurnout  MonthlyIncome_X_IndiceDeSatisfacaoGeral  \
0                          35958.0         

In [12]:
# 2. Selecionar as colunas do DataFrame Pandas
df_features = df_final_features.copy() # .copy() é uma boa prática para evitar SettingWithCopyWarning

# 3. Definir o nome do arquivo CSV de saída
output_csv_filename = "sv_hr_final_features_v1.csv"

# 4. Salvar o DataFrame como CSV
# Para salvar no ambiente temporário do Colab (será apagado ao encerrar a sessão):
df_features.to_csv(output_csv_filename, index=False) # index=False para não escrever o índice do DataFrame como uma coluna no CSV

print(f"DataFrame salvo como '{output_csv_filename}' no ambiente temporário do Colab.")
print(df_features.head()) # Exibe as primeiras linhas do DataFrame salvo

# --- OPÇÃO RECOMENDADA: Salvar no Google Drive para persistência ---
from google.colab import drive
import os

# 1. Montar seu Google Drive (se ainda não o fez)
# drive.mount('/content/drive')

# 2. Definir o caminho completo no Google Drive
# Crie uma pasta 'dados_output' no seu Drive se não existir
output_drive_path = '/content/drive/MyDrive/dev/datascienceexp/'
os.makedirs(output_drive_path, exist_ok=True) # Cria a pasta se ela não existir

full_output_path_drive = os.path.join(output_drive_path, output_csv_filename)

# 3. Salvar o DataFrame como CSV no Google Drive
df_features.to_csv(full_output_path_drive, index=False)

print(f"\nDataFrame salvo também no Google Drive em: '{full_output_path_drive}'")


DataFrame salvo como 'sv_hr_final_features_v1.csv' no ambiente temporário do Colab.
   MaritalStatus_TargetEncoded  IndiceDeBurnout^2  MonthlyIncome^2  \
0                     0.255319               36.0       35916049.0   
1                     0.124814               25.0       26316900.0   
2                     0.255319               25.0        4368100.0   
3                     0.124814               25.0        8462281.0   
4                     0.124814               25.0       12027024.0   

   MonthlyIncome_X_IndiceDeBurnout  YearsAtCompany_X_IndiceDeBurnout  \
0                          35958.0                              36.0   
1                          25650.0                              50.0   
2                          10450.0                               0.0   
3                          14545.0                              40.0   
4                          17340.0                              10.0   

   MonthlyIncome_X_IndiceDeSatisfacaoGeral  \
0               

# Justificativas e Impactos das 10 Features Avançadas

Grupo 1: Features de Target Encoding
Estas features convertem categorias de texto em um número que representa o risco de turnover associado àquela categoria.

1. JobRole_TargetEncoded (Cargo Codificado pelo Alvo)

Justificativa Técnica: A variável JobRole é categórica e possui muitas classes (alta cardinalidade). O Target Encoding a transforma em uma única variável numérica, onde o valor de cada cargo é diretamente proporcional à sua taxa de turnover. Isso cria um sinal forte e monotônico para o modelo.


Justificativa de Negócio e Impacto Esperado: A análise exploratória mostrou que cargos como "Representante de Vendas" e "Técnico de Laboratório" têm maior propensão ao turnover.

Impacto Esperado: O modelo aprenderá a atribuir um peso de risco maior a funcionários nesses cargos específicos, permitindo que o RH foque ações de retenção em funções mais vulneráveis.

2. MaritalStatus_TargetEncoded (Estado Civil Codificado pelo Alvo)

Justificativa Técnica: Converte a variável demográfica MaritalStatus em um valor numérico que reflete o risco. É mais eficiente que One-Hot Encoding e captura a ordem de risco (se houver) entre as categorias.


Justificativa de Negócio e Impacto Esperado: Foi observado que a proporção de solteiros no grupo de turnover é 50% maior. Isso sugere que podem ter menor vínculo com a empresa.

Impacto Esperado: O modelo poderá identificar perfis demográficos que, estatisticamente, possuem maior mobilidade no mercado de trabalho e menor barreira para mudar de emprego.

3. EducationField_TargetEncoded (Área de Formação Codificada pelo Alvo)

Justificativa Técnica: Similar às anteriores, resume o risco associado a cada campo de educação em um único número, simplificando a complexidade para o modelo.


Justificativa de Negócio e Impacto Esperado: As áreas de Marketing, Técnico de TI e RH apresentaram maior atrito.

Impacto Esperado: Esta feature permite que o modelo identifique se a formação do colaborador pertence a uma área com alta demanda no mercado ou com desafios internos específicos na empresa, ajustando o risco de saída.

Grupo 2: Features de Potência (Não-Linearidade)
Estas features ajudam o modelo a capturar relações onde o efeito de uma variável se acelera.

4. IndiceDeBurnout^2 (Índice de Burnout ao Quadrado)

Justificativa Técnica: A relação entre o burnout e a decisão de sair provavelmente não é linear. O termo quadrático permite ao modelo capturar um efeito acelerado, onde um aumento de 1 ponto no índice de burnout em níveis já altos tem um impacto muito maior do que em níveis baixos.

Justificativa de Negócio e Impacto Esperado: Um funcionário com burnout "nível 8" não tem apenas o dobro do risco de um com "nível 4"; o risco pode ser exponencialmente maior. Impacto Esperado: O modelo se tornará mais sensível para identificar funcionários que ultrapassaram um "ponto de inflexão" crítico, sinalizando uma necessidade de intervenção urgente.

5. MonthlyIncome^2 (Salário Mensal ao Quadrado)

Justificativa Técnica: Ajuda a modelar o efeito não-linear do salário. O impacto de um aumento de R1.000
e
ˊ
 muitomaiorparaquemganhaR2.000 do que para quem ganha R$15.000.


Justificativa de Negócio e Impacto Esperado: A análise mostrou que salários baixos são um forte motivador para a saída.

Impacto Esperado: O modelo poderá entender melhor a "utilidade marginal decrescente" do salário, ou seja, que a insatisfação com a remuneração é muito mais acentuada nas faixas salariais mais baixas.

Grupo 3: Features de Interação (Efeitos Combinados)
Estas são as features mais sofisticadas, pois medem como o efeito de uma variável muda na presença de outra.

6. MonthlyIncome_X_IndiceDeBurnout (Interação Salário vs. Burnout)

Justificativa Técnica: Cria uma variável que representa o efeito combinado do salário e do esgotamento.

Justificativa de Negócio e Impacto Esperado: Estar sobrecarregado é ruim. Estar sobrecarregado e se sentir mal pago é uma combinação explosiva. Impacto Esperado: O modelo poderá identificar com altíssima precisão o perfil de funcionário em maior risco: aquele que sente que seu esforço (burnout) não é recompensado financeiramente.

7. YearsAtCompany_X_IndiceDeBurnout (Interação Tempo de Casa vs. Burnout)

Justificativa Técnica: Testa se o impacto do burnout depende do tempo de casa do funcionário.

Justificativa de Negócio e Impacto Esperado: Um funcionário novo com alto burnout pode desistir rapidamente. Um funcionário antigo com alto burnout pode estar em seu limite final após anos de dedicação. Impacto Esperado: Ajuda o modelo a diferenciar o risco, permitindo ações distintas: para o novo, uma integração melhor; para o antigo, talvez uma reavaliação de carreira ou cargo.

8. MonthlyIncome_X_IndiceDeSatisfacaoGeral (Interação Salário vs. Satisfação)

Justificativa Técnica: Mede se o dinheiro pode "compensar" uma baixa satisfação geral com o ambiente e as relações.

Justificativa de Negócio e Impacto Esperado: Um funcionário com alta satisfação talvez tolere um salário menor. Um funcionário insatisfeito talvez não seja retido nem com um salário alto. Impacto Esperado: Permite ao modelo avaliar se as estratégias de retenção devem focar em remuneração ou em melhorias no ambiente, dependendo do perfil do funcionário.

9. YearsAtCompany_X_IndiceDeSatisfacaoGeral (Interação Tempo de Casa vs. Satisfação)

Justificativa Técnica: Avalia como a "paciência" com a insatisfação muda ao longo do tempo.


Justificativa de Negócio e Impacto Esperado: A fase de 3 a 5 anos é crítica. Estar insatisfeito após 5 anos pode ser um sinal de "promessas não cumpridas" e um preditor de saída mais forte do que a insatisfação no primeiro ano.

Impacto Esperado: O modelo poderá dar mais peso à insatisfação de funcionários mais antigos, que já deram tempo para a empresa "provar seu valor".

10. IndiceDeBurnout_X_IndiceDeSatisfacaoGeral (Interação Burnout vs. Satisfação Geral)

Justificativa Técnica: Testa a interação entre o esgotamento individual (ligado ao trabalho) e a satisfação com o ambiente (ligado ao contexto social e físico).

Justificativa de Negócio e Impacto Esperado: Um funcionário pode estar sobrecarregado, mas se ele gosta de seus colegas e do ambiente, isso pode funcionar como um fator de proteção.

Impacto Esperado: O modelo poderá identificar se um ambiente de trabalho positivo e boas relações interpessoais  são capazes de mitigar o risco de burnout, informando ao RH que investir em cultura pode ser um "antídoto" eficaz
